In [2]:
#Import Libraries and Package Modules

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

#Defining Contants
BATCH_SIZE = 128
EPOCHS = 10
LEARNING_RATE = 0.001

#Defining a Function to Download MNIST Dataset
def downloadDatasets():
    #Dataset for Training the Model
    trainData = datasets.MNIST(
        root="data",
        train=True,
        download=True,
        transform=ToTensor(),
    )
    #Dataset for Testing/ Validating the Model
    validationData = datasets.MNIST(
        root="data",
        train=False,
        download=True,
        transform=ToTensor(),
    )
    return trainData, validationData

def createDataLoader(trainData, batchSize):
    trainDataloader = DataLoader(trainData, batch_size=batchSize)
    return trainDataloader

#Built a Model - Feed Forward Network
class FeedForwardNet(nn.Module):
    #Creating Constructor
    def __init__(self):
        #Invoke Cnstructor of base class
        super().__init__()
        #Create Initial Layer
        self.flatten = nn.Flatten()
        #Create Dense Layers
        #Use Sequential() to pack up Dense layers together
        self.denseLayers = nn.Sequential(
            #Appling Linear Layer
            nn.Linear(28 * 28, 256),
            #Applying Activation Function: Rectified Linear Unit
            nn.ReLU(),
            #Another Linear Function
            nn.Linear(256, 10)
        )
        #Perform Basic Transformation using softmax()
        self.softmax = nn.Softmax(dim=1)
    #Defining Forward Function
    def forward(self, input_data):
        #Get Flattened Data
        x = self.flatten(input_data)
        #Get Logits
        logits = self.denseLayers(x)
        #Get Predictions
        predictions = self.softmax(logits)
        return predictions


def trainOneEpoch(model, dataLoader, lossFn, optimiser, device):
    for input, target in dataLoader:
        input, target = input.to(device), target.to(device)
        # calculate loss
        prediction = model(input)
        loss = lossFn(prediction, target)
        # backpropagate error and update weights
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
    print(f"Loss: {loss.item()}")
    
def train(model, dataLoader, lossFn, optimiser, device, epochs):
    for i in range(epochs):
        print(f"Epoch {i+1}")
        trainOneEpoch(model, dataLoader, lossFn, optimiser, device)
        print("---------------------------")
    print("Finished training")
    
       

#Updating Script to Check Feed Forward Network Model Working
if __name__ == "__main__":

    # download data and create data loader
    trainData, _ = downloadDatasets()
    trainDataLoader = createDataLoader(trainData, BATCH_SIZE)

    #Construct Model and Assign it to device
    if torch.cuda.is_available():
        device = "cuda"
    else:
        device = "cpu"
    print(f"Using {device}")
    feedForwardNet = FeedForwardNet().to(device)
    print(feedForwardNet)

    #Initialise loss funtion + optimiser
    lossFn = nn.CrossEntropyLoss()
    optimiser = torch.optim.Adam(feedForwardNet.parameters(),
                                 lr=LEARNING_RATE)

    #Train Model
    train(feedForwardNet, trainDataLoader, lossFn, optimiser, device, EPOCHS)

    #Save Model
    torch.save(feedForwardNet.state_dict(), "feedforwardnet.pth")
    print("Trained feed forward net saved at feedforwardnet.pth")

Using cpu
FeedForwardNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (denseLayers): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=10, bias=True)
  )
  (softmax): Softmax(dim=1)
)
Epoch 1
Loss: 1.515464425086975
---------------------------
Epoch 2
Loss: 1.4956187009811401
---------------------------
Epoch 3
Loss: 1.491089940071106
---------------------------
Epoch 4
Loss: 1.4868050813674927
---------------------------
Epoch 5
Loss: 1.477141261100769
---------------------------
Epoch 6
Loss: 1.4740997552871704
---------------------------
Epoch 7
Loss: 1.4738730192184448
---------------------------
Epoch 8
Loss: 1.4728350639343262
---------------------------
Epoch 9
Loss: 1.4728622436523438
---------------------------
Epoch 10
Loss: 1.4735342264175415
---------------------------
Finished training
Trained feed forward net saved at feedforwardnet.pth


In [6]:

classMapping = [
    "0",
    "1",
    "2",
    "3",
    "4",
    "5",
    "6",
    "7",
    "8",
    "9"
]


def predict(model, input, target, class_mapping):
    model.eval()
    with torch.no_grad():
        predictions = model(input)
        # Tensor (1, 10) -> [ [0.1, 0.01, ..., 0.6] ]
        predictedIndex = predictions[0].argmax(0)
        predicted = class_mapping[predictedIndex]
        expected = class_mapping[target]
    return predicted, expected


if __name__ == "__main__":
    # load back the model
    feedForwardNet = FeedForwardNet()
    stateDict = torch.load("feedforwardnet.pth")
    feedForwardNet.load_state_dict(stateDict)

    # load MNIST validation dataset
    _, validationData = downloadDatasets()

    # get a sample from the validation dataset for inference
    input, target = validationData[0][0], validationData[0][1]

    # make an inference
    predicted, expected = predict(feedForwardNet, input, target,
                                  classMapping)
    print(f"Predicted: '{predicted}', expected: '{expected}'")

Predicted: '7', expected: '7'
